In [ ]:
import os

# Rename CUSTOM_MODEL_NAME
### Make Sure CUSTOM_MODEL_NAME Is The Name Of The Model You Want To Run

In [ ]:
# CUSTOM_MODEL_NAME Variable Should Be The Same Name As The Trained Model You Want To Run
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

##### 

# Setup Paths

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

##### 

# Load Trained Model From Checkpoint

### This Loads Your Trained Model

In [ ]:
import object_detection

In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

### On Line 7 Change The Number In 'ckpt-3' To The Number Of The Highest Checkpoint In Your Model Directory Located In: Tensorflow > Workspace > Models > CUSTOM_MODEL_NAME

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

##### 

# Import Dependencies

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

# 

# <center>Detect From Image Or Webcam Below</center>

##### 

# Detect From An Image

### Change JPG Image Name In IMAGE_PATH To Name Of An Image In The Test Folder
### Images Located At: Tensorflow > Workspace > Images > Test

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'thumbsdown.0b213414-166e-11ec-a6fd-acde48001122.jpg')

### Run To Test Image Detection

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=1,
            min_score_thresh=.7,  # Change This For Min. Score % To Show Box
            line_thickness=16,
#            skip_scores=True,
#            skip_labels=True,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

# 

# Real Time Detections from your Webcam

### Run To Test Live Detection
### Click On Window And Press "q" To Close When Done
### Change Line 31 For Min. % To Show Detection
## May Take Time To Open

In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=2,
                min_score_thresh=.7, # Change This For Min. Score % To Show Box
                line_thickness=8,
#               skip_scores=True,
#               skip_labels=True,
                agnostic_mode=False)

    cv2.imshow('object detection', image_np_with_detections)
    cv2.setWindowProperty('object detection', cv2.WND_PROP_TOPMOST, 1)
    cv2.moveWindow('object detection', 400, 30)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break

### If You Are Happy With Your Models Performance You Can Go To Notebook "4-Export-Object-Detection-Model" And Export Your Model To Different Formats

##### 

### If Your Model Does Not Perform Well, You Have 3 Choices To Potentially Get Better Results:
### 1) Train Your Model For More Steps
### 2) Add More Images Of Annotated Objects
### 3) Use A Different Tensorflow Pre-Trained Model To Transfer Learn From

##### 

### If You Would Like To Train This Model For More Steps Go To Notebook "Performance-Continue-To-Train-Model"
### If You Would Like To Add More Images To Train On Go To Notebook "Performance-Add-Images-To-Trained-Model" And Then Continue To Train Your Model At "Performance-Continue-To-Train-Model"
### If You Would Like To Try Another Pre-Trained Model I Would Suggest To Start Over(re-clone the repository) In A New Project Directory, Although It Can Be Done Here. Just Copy Your "train" and "test" Image Folders Over Into Your New Project.